In [1]:
## Importing libraries 
import nemo
import nemo.collections.asr as nemo_asr
import jiwer
from jiwer import wer
import json
import zipfile
import os 
import librosa
import torch
import torch.nn as nn
import copy
import pytorch_lightning as pl
import time 
import torchaudio
import noisereduce as nr
from scipy.io import wavfile
from omegaconf import OmegaConf, open_dict
from nemo.collections.asr.models.ctc_bpe_models import EncDecCTCModelBPE
from nemo.core.config import hydra_runner
from nemo.utils import logging
from nemo.utils.exp_manager import exp_manager

[NeMo W 2022-05-24 15:39:54 optimizers:55] Apex was not found. Using the lamb or fused_adam optimizer will error out.
################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################



In [3]:
audio_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/audio_files_train/'
script_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/train_script.txt'
train_manifest_path = '/home/dgxuser/Desktop/kailin/location_data (updated)/train_manifest.json'

audio_paths_train = []
n = 1 
while os.path.exists(audio_dir + str(n).zfill(4) + '.wav'):
    # resample at 16kHz
    y, sr = torchaudio.load(audio_dir + str(n).zfill(4) + '.wav')
    y = y.mean(dim=0) # if there are multiple channels, average them to single channel
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        y_resampled = resampler(y)
    torchaudio.save(audio_dir + str(n).zfill(4) + '_resampled.wav', y.unsqueeze(0), sr)

    rate, data = wavfile.read(audio_dir + str(n).zfill(4) + '_resampled.wav')
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    wavfile.write(audio_dir + str(n).zfill(4) + '_clean.wav', rate, reduced_noise)
    audio_paths_train.append(audio_dir + str(n).zfill(4) + '_clean.wav')
    n = n+1

with open(train_manifest_path, 'w') as fout: 
    with open(script_dir, encoding = 'utf-8') as script:
        for path in audio_paths_train: 
            line = script.readline().strip().lower()
            duration = librosa.core.get_duration(filename=path)
            metadata = {
                "audio_filepath": path,
                "duration": duration,
                "text": line
            }
            json.dump(metadata, fout)
            fout.write('\n')    

In [4]:
audio_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/audio_files_val/'
script_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/val_script.txt'
val_manifest_path = '/home/dgxuser/Desktop/kailin/location_data (updated)/val_manifest.json'

audio_paths_val = []
n = 1 
while os.path.exists(audio_dir + str(n).zfill(4) + '.wav'):
    # resample at 16kHz
    y, sr = torchaudio.load(audio_dir + str(n).zfill(4) + '.wav')
    y = y.mean(dim=0) # if there are multiple channels, average them to single channel
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        y_resampled = resampler(y)
    torchaudio.save(audio_dir + str(n).zfill(4) + '_resampled.wav', y.unsqueeze(0), sr)

    rate, data = wavfile.read(audio_dir + str(n).zfill(4) + '_resampled.wav')
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    wavfile.write(audio_dir + str(n).zfill(4) + '_clean.wav', rate, reduced_noise)
    audio_paths_val.append(audio_dir + str(n).zfill(4) + '_clean.wav')
    n = n+1

with open(val_manifest_path, 'w') as fout: 
    with open(script_dir, encoding = 'utf-8') as script:
        for path in audio_paths_val: 
            line = script.readline().strip().lower()
            duration = librosa.core.get_duration(filename=path)
            metadata = {
                "audio_filepath": path,
                "duration": duration,
                "text": line
            }
            json.dump(metadata, fout)
            fout.write('\n')    

In [5]:
audio_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/audio_files_test/'
script_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/test_script.txt'
test_manifest_path = '/home/dgxuser/Desktop/kailin/location_data (updated)/test_manifest.json'

audio_paths_test = []
n = 1 
while os.path.exists(audio_dir + str(n).zfill(4) + '.wav'):
    # resample at 16kHz
    y, sr = torchaudio.load(audio_dir + str(n).zfill(4) + '.wav')
    y = y.mean(dim=0) # if there are multiple channels, average them to single channel
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(sr, 16000)
        y_resampled = resampler(y)
    torchaudio.save(audio_dir + str(n).zfill(4) + '_resampled.wav', y.unsqueeze(0), sr)

    rate, data = wavfile.read(audio_dir + str(n).zfill(4) + '_resampled.wav')
    # perform noise reduction
    reduced_noise = nr.reduce_noise(y=data, sr=rate)
    wavfile.write(audio_dir + str(n).zfill(4) + '_clean.wav', rate, reduced_noise)
    audio_paths_test.append(audio_dir + str(n).zfill(4) + '_clean.wav')
    n = n+1

with open(test_manifest_path, 'w') as fout: 
    with open(script_dir, encoding = 'utf-8') as script:
        for path in audio_paths_test: 
            line = script.readline().strip().lower()
            duration = librosa.core.get_duration(filename=path)
            metadata = {
                "audio_filepath": path,
                "duration": duration,
                "text": line
            }
            json.dump(metadata, fout)
            fout.write('\n')    

In [22]:
# Initialise quartznet model 
quartznet = nemo_asr.models.EncDecCTCModel.from_pretrained(model_name="stt_en_quartznet15x5")

[NeMo I 2022-05-24 15:55:48 cloud:56] Found existing object /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-05-24 15:55:48 cloud:62] Re-using file from: /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-05-24 15:55:48 common:747] Instantiating model from pre-trained checkpoint


[NeMo W 2022-05-24 15:55:49 modelPT:148] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-05-24 15:55:49 modelPT:155] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-05-24 15:55:49 features:259] PADDING: 16
[NeMo I 2022-05-24 15:55:49 features:276] STFT using torch
[NeMo I 2022-05-24 15:55:49 save_restore_connector:209] Model EncDecCTCModel was successfully restored from /home/dgxuser/.cache/torch/NeMo/NeMo_1.8.2/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.


In [23]:
# test before training 
script_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/test_script.txt'
predicted = []
transcript = []
with open(script_dir, encoding = 'utf-8') as script:
    for path in audio_paths_test: 
        line = script.readline().strip()
        files = [path]
        transcript.append(line)
        pred_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(pred_line)
        print(f'{line} was predicted as {pred_line}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

bishan bishan was predicted as beshan bishan


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

bishan bishan was predicted as besun be sun


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

yishun yishun was predicted as you should yo should


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

yishun yishun was predicted as yo shouldn't yishould


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

yishun yishun was predicted as he shouldn't be sure


In [24]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))

Word Error Rate: 1.1521739130434783


In [25]:
def enable_bn_se(m):
    if type(m) == nn.BatchNorm1d:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

    if 'SqueezeExcite' in type(m).__name__:
        m.train()
        for param in m.parameters():
            param.requires_grad_(True)

quartznet.encoder.freeze()
quartznet.encoder.apply(enable_bn_se)
print("Model encoder has been frozen")

Model encoder has been frozen


In [26]:
quartznet.decoder.freeze()
quartznet.decoder.apply(enable_bn_se)
print("Model decoder has been frozen, and batch normalization has been unfrozen")

Model decoder has been frozen, and batch normalization has been unfrozen


In [27]:
## Config Information 
try:
    from ruamel.yaml import YAML
except ModuleNotFoundError:
    from ruamel_yaml import YAML
config_path = './configs/config.yaml'

yaml = YAML(typ='safe')
with open(config_path) as f:
    params = yaml.load(f)
print(params)

{'name': 'QuartzNet15x5', 'sample_rate': 16000, 'repeat': 1, 'dropout': 0.0, 'separable': True, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'model': {'train_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 'validation_ds': {'manifest_filepath': '???', 'sample_rate': 16000, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'batch_size': 32, 'shuffle': False}, 'preprocessor': {'_target_': 'nemo.collections

In [28]:
# add in manifest paths 
params['model']['train_ds']['manifest_filepath'] = train_manifest_path
params['model']['validation_ds']['manifest_filepath'] = train_manifest_path

# Point to the new validation data for fine-tuning
quartznet.setup_validation_data(val_data_config=params['model']['validation_ds'])

# Point to the data we'll use for fine-tuning as the training set
quartznet.setup_training_data(train_data_config=params['model']['train_ds'])

# Create a PyTorch Lightning trainer and call `fit` again.
trainer = pl.Trainer(max_epochs=2, accelerator='gpu', devices=1)
trainer.fit(quartznet)

[NeMo I 2022-05-24 15:57:25 collections:186] Dataset loaded with 177 files totalling 0.15 hours
[NeMo I 2022-05-24 15:57:25 collections:187] 0 files were filtered totalling 0.00 hours
[NeMo I 2022-05-24 15:57:25 collections:186] Dataset loaded with 177 files totalling 0.15 hours
[NeMo I 2022-05-24 15:57:25 collections:187] 0 files were filtered totalling 0.00 hours


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
[NeMo W 2022-05-24 15:57:25 modelPT:475] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2022-05-24 15:57:25 modelPT:587] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )


[NeMo W 2022-05-24 15:57:25 lr_scheduler:816] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !

  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 18.9 M
2 | decoder           | ConvASRDecoder                    | 29.7 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
77.3 K    Trainable params
18.8 M    Non-trainable params
18.9 M    Total params
75.698    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [29]:
# test after training 
script_dir = '/home/dgxuser/Desktop/kailin/location_data (updated)/test_script.txt'
predicted = []
transcript = []
with open(script_dir, encoding = 'utf-8') as script:
    for path in audio_paths_test: 
        line = script.readline().strip()
        files = [path]
        transcript.append(line)
        pred_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(pred_line)
        print(f'{line} was predicted as {pred_line}')

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

bishan bishan was predicted as bishan bishan


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

bishan bishan was predicted as bishan bishan


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

yishun yishun was predicted as ishun ishan


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

yishun yishun was predicted as ishulan ishan


Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

yishun yishun was predicted as ishon bishun


In [30]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))

Word Error Rate: 0.8913043478260869


In [31]:
# test WER after tuning 
data_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/WAVE'
script_dir = '/home/dgxuser/Desktop/kailin/CHANNEL0/SCRIPT/'
speakers = ['0010']
transcript = []
predicted = []
count = 0

for i in speakers: 
  path_unzipped = data_dir + '/SPEAKER' + i
  if not os.path.exists(path_unzipped):
    path = data_dir + '/SPEAKER' + i + '.zip'
    zip_ref = zipfile.ZipFile(path, 'r')
    zip_ref.extractall(path=data_dir)
  n = 0 
  wav_folder = data_dir + '/SPEAKER' + i + '/SESSION'
  while os.path.exists(wav_folder + str(n)):
    files = []
    for filename in os.listdir(wav_folder + str(n)):
      files.append(filename)
    files = sorted(files)
    script_path = script_dir + filename[:6] + '.TXT'
    with open(script_path, encoding = 'utf-8') as script:
      for filename in files:
        script.readline()
        line = script.readline().strip()
        words = line.split()
        edited_words = []
        for word in words:
          if (word != '<SPK/>') and (word != '**'):
            edited_words.append(word.lower())
        transcript_line = ""
        for word in edited_words: 
          transcript_line = transcript_line + word + " "
        transcript_line = transcript_line[:-1]
        transcript.append(transcript_line)
        wav_path = os.path.join(wav_folder + str(n), filename)
        files = [wav_path]
        predicted_line = quartznet.transcribe(paths2audio_files=files)[0]
        predicted.append(predicted_line)
        count = count+1
        #print('"' + transcript_line + '" was predicted as "' + predicted_line + '"')
    n = n+1

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

...

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s]

In [32]:
error = wer(transcript, predicted)
print('Word Error Rate: ' + str(error))
print('Number of Testcases (Lines): ' + str(count))

Word Error Rate: 0.1198792583009918
Number of Testcases (Lines): 380
